# `BUILDING A ML MODEL TO PREDICT CUSTOMER CHURN` 
#### Using the CRISP-DM framework

## `Business Understanding`


#### Goal/Description
To create a machine learning model to predict the likelihood of retaining a customer

#### `Null Hypothesis`
There is no relationship between a tech savy customer and the customer retention

#### `Alternate Hypothesis`
There is a relationship between a tech savy customer and the customer retention

#### `Analytical Questions`
- How does tenure and monthly charge affect customer churn?
- What is the likelihood of a customer with online security and protection to churn?
- What is the relationship between the type of contract and the likelihood of a customer churn?
- Do customers with dependents and internet security likely to Churn?

## `Data Understanding`

#### Data Source
The data was sourced from a Telecommunication company and divided into three (3) parts :
- 3000 rows as the training data
- 2000 rows as the evaluation data 
- 2000 rows as the test data 

### `Issues`
- Some columns have multiple adjectives of the same word. eg no,no internet service,false 


#### Data Exploration

##### `Libraries`

In [ ]:
#Libraries imported
import sqlalchemy as sa
import pyodbc  
from dotenv import dotenv_values 
import pandas as pd
from scipy import stats 
from scipy.stats import kruskal
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns
import collections
import numpy as np
import warnings


###FILTER WARNINGS

from sklearn.model_selection import * #train_test_split, cross_val_score

from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.utils import resample
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC 
from catboost import CatBoostClassifier
import xgboost as xgb
from xgboost import XGBClassifier


from sklearn.metrics import *
from sklearn.model_selection import * 
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression


##### `Database Connection`

In [ ]:
#Access protocols for the SQL Database
env_variables= dotenv_values('logins.env')
database= env_variables.get('database')
server = env_variables.get('server')
username = env_variables.get('username')
password = env_variables.get('password')

In [ ]:
#Creation of Connection to Database
connection_string = f"DRIVER={{SQL Server}};SERVER={server};DATABASE={database};UID={username};PWD={password};MARS_Connection=yes;MinProtocolVersion=TLSv1.2;"
connection = pyodbc.connect(connection_string)

In [ ]:
#Querying SQL Database and reading the table into a dataframe
query = "SELECT * FROM LP2_Telco_churn_first_3000"

sql_df= pd.read_sql(query, connection)
sql_df.info()

In [ ]:
# Describing the SQL Dataframe
sql_df.describe(include='all').T

##### `Accessing the second set of data in CSV format`

In [ ]:
##Accessing the second set of data 
csv_df = pd.read_csv("data\\LP2_Telco-churn-second-2000.csv")
csv_df.info()

In [ ]:
# Describing the Dataframe
csv_df.describe(include='all').T

##### `Merging the Two Dataframes`

In [ ]:
com_df=pd.concat([sql_df,csv_df],ignore_index=True)
com_df.head(5)
com_df.shape

In [ ]:
#Checking the datatypes of the columns
datatypes = com_df.dtypes
datatypes

##### Converting the TotalCharges datatype to float64

In [ ]:
#Converting TotalCharges column to numeric
com_df['TotalCharges'] = pd.to_numeric(com_df['TotalCharges'], errors='coerce')
com_df=com_df.reset_index()

In [ ]:
# Checking the Null value
com_df.isnull().sum()

In [ ]:
com_df.head(5)

In [ ]:
#Dropping the index column
com_df = com_df.drop(['index'], axis=1)

##### Replacing all negatives with False and positives with True

In [ ]:
com_df.replace(['No','No internet service','false','No phone service'], "False", inplace=True)

com_df.replace('Yes',"True", inplace=True)



In [ ]:
com_df['SeniorCitizen'] = np.where(com_df['SeniorCitizen'] == 1, True, False)


In [ ]:
com_df.InternetService.replace('false','None')

In [ ]:
datatypes = com_df.dtypes
datatypes

##### Making the True/False to Boolean

In [ ]:
com_df.replace({'True': True, 'False': False}, inplace=True)

In [ ]:
com_df.to_csv("data/customer_churn_merged")

### Univariate Analysis

In [ ]:
# Distribution of the variables
com_df.hist(density=True,figsize=(20, 15), facecolor='lightgreen', alpha=0.75,grid=False)

plt.show()

In [ ]:
# Visualize the distribution of categorical columns
categoricals = [column for column in com_df.columns if com_df[column].dtype == "O"]
for column in categoricals:
        if column not in ['customerID']:
                fig = px.histogram(com_df, x=com_df[column], text_auto=True,color=column,
                               title=f"Distribution of customers based on {column}")
                fig.update_layout(uniformtext_minsize=8, uniformtext_mode='hide', xaxis_tickangle=-45)
                fig.show()


#### OBSERVATION
- The Gender is evenly distributed 
- Over 50% of all contract types are month-on-month basis
- Electronic Check is the most used,covering 30% of all payment methods


In [ ]:
fig = plt.figure(figsize =(5, 4))
 
# Creating plot
plt.boxplot(com_df.tenure)
plt.show()

In [ ]:
fig = plt.figure(figsize =(5, 4))
 
# Creating plot
plt.boxplot(com_df.MonthlyCharges)
plt.show()

### Bivariate Analysis

In [ ]:
# Summarizing the relationships between the variables with a heatmap of the correlations
correlation_matrix = com_df.corr(numeric_only= True)
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True,cmap='vlag')
plt.title("Correlation heatmap of the Telecom Dataset")
plt.show()

 ## `Answering the Analytical Questions`


##### `How does tenure and monthly charge affect customer churn?`


In [ ]:

bins = [ 10, 30, 50,70]
df=com_df
labels = ['Newbie', 'Young', 'Oldies']
df['tenure Group'] = pd.cut(df['tenure'], bins=bins, labels=labels)
streamers = com_df.groupby(['tenure Group','Churn'])['MonthlyCharges'].mean().sort_values(ascending=True)
#sns.displot(streamers, x="tenure Group",hue="Churn", element="step")
streamers.plot(kind='bar', title = 'How does tenure and monthly charge affect customer churn', figsize = (10,6), cmap='Dark2', rot = 30);
#streamers.plot(kind='bar')
plt.show()


#### OBSERVATION
- New,Existing and Old Customers with higher charges for software usage are the ones churning.
- There has to be a loyalty promotion for old customers to lock in the old customers.
- There can also be a signup discount to new customers to lock them in on the software.

##### `What is the likelihood of a customer with online security and device protection to churn?`


In [ ]:
cust_retention = com_df.groupby(['OnlineSecurity','DeviceProtection'])['Churn'].count().sort_values(ascending=True)
cust_retention.plot(kind='bar', title = 'The likelihood of a customer with online security and device protection to churn', figsize = (10,6), cmap='Dark2', rot = 30)

#### OBSERVATION
- Customers with no security at all are more likely to Churn. 
- Basic cybersecurity can be done to curb customer doubt to reduce Churn.

#### `What is the relationship between the type of contract and the likelihood of a customer churn?`


In [ ]:
cust_contract = com_df.groupby('Contract')['Churn'].count().sort_values(ascending=True)
cust_contract.plot(kind='bar', title = 'The relationship between the type of contract and the likelihood of a customer churn', figsize = (10,6), cmap='Dark2', rot = 30)

#### OBSERVATION
- Month-to-Month Customers are more likely to churn as they are likely to be floating users.

#### `Do customers with dependents and internet security likely to Churn?`

In [ ]:
cust_contract = com_df.groupby(['OnlineSecurity','Dependents'])['Churn'].count().sort_values(ascending=True)
cust_contract.plot(kind='bar', title = 'Do customers with dependents and internet security likely to Churn?', figsize = (10,6), cmap='Dark2', rot = 30)

#### OBSERVATION
Customers with both Online Security and Dependents are less likely to churn.

In [ ]:
com_df.isnull().sum()


In [ ]:
#Dropping Empty rows
com_df = com_df.dropna(subset=['OnlineSecurity','OnlineBackup','DeviceProtection','MultipleLines','TotalCharges','Churn'],axis=0)

In [ ]:
#finding duplicates
duplicate = com_df[com_df.duplicated()]
duplicate.shape

##### OBSERVATION 
No duplicates found

#### `HYPOTHESIS`

In [ ]:
#Checking Normality of the data 

def check_normality(data,name):
    test_stat_normality, p_value_normality=stats.shapiro(data)
    print("p value:%.20f" % p_value_normality)
    if p_value_normality <0.05:
        print(f"Reject null hypothesis >> The data for {name} is not normally distributed")
    else:
        print(f"Fail to reject null hypothesis >> The data for {name} is normally distributed")

In [ ]:
#Hypothesis

df_tech= com_df.loc[com_df.OnlineSecurity & com_df.DeviceProtection]
online=com_df.loc[com_df.OnlineSecurity]
device= com_df.loc[com_df.DeviceProtection]


In [ ]:
#Normality Checks
check_normality(df_tech.TotalCharges,'Online Security and Device Protection')
check_normality(online.TotalCharges,'Online Security')
check_normality(device.TotalCharges,'Device Protection')

In [ ]:
#Using the P-Levene to test the Hypothesis
stat, pvalue_levene= stats.levene(df_tech.TotalCharges, online.TotalCharges,device.TotalCharges )

print("p value:%.10f" % pvalue_levene)
if pvalue_levene <0.05:
    print("Reject null hypothesis >> The variances of the samples are different.")
else:
    print("Fail to reject null hypothesis >> The variances of the samples are same.")

##### Observation 
- Data samples are not normally distributed
- The variances of the samples are different
- Therefore a Non-Parametric test must be done (Kruskal Test)

In [ ]:
#Kruskal Test

stat, p = kruskal(df_tech.TotalCharges, online.TotalCharges,device.TotalCharges)
print('Statistics=%.3f, p=%.15f' % (stat, p))

if p > 0.05:
 print('All sample distributions are the same (fail to reject H0)')
else:
 print('One or more sample distributions are not equal distributions (reject null Hypothesis)')

##### OBSERVATION
Reject the null Hypothesis

### `Data preparation`

#### Feature Correlation and Selection

In [ ]:
# Summarize the relationships between the variables with a heatmap of the correlations
correlation_matrix = df.corr(numeric_only= True).round(3)
mask = np.triu(np.ones_like(correlation_matrix, dtype=bool))
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True,cmap='vlag',mask=mask)
plt.title("Correlation heatmap of the dataset")
plt.show()

In [ ]:
df =com_df.drop(columns=['customerID','gender','TotalCharges','tenure Group'],axis=1)
df.head(1)

In [ ]:
# Dropping row with null value
df.dropna(axis=0, inplace=True)

In [ ]:
df.replace({True: 'Yes', False: 'No'})
df.head(1)

In [ ]:
def str_convert(df,column_name):
    df[column_name]=df[column_name].replace({True: 'True', False: 'False'})

    return df

In [ ]:
df=str_convert(df,"PaperlessBilling")
df=str_convert(df,"PhoneService")
df=str_convert(df,"SeniorCitizen")
df=str_convert(df,"Partner")
df=str_convert(df,"InternetService")
df=str_convert(df,"Dependents")

In [ ]:
df["Churn"].unique().tolist()
df["Churn"]=df["Churn"].replace({True: '1', False: '0'})

#### `Distribution of the dependent variable`

##### STRATIFICATION ZONE

In [ ]:
# Separate majority and minority classes
df_stay = df[df.Churn=='0']
df_left = df[df.Churn=='1']

print((len(df_stay)/len(df)),(len(df_left)/len(df)))
print(len(df_left))

##### Observation
- About 70% of the customers stayed as compared to the customers that left therefore the churned customers represent the minority group

In [ ]:
# Downsample minority class
df_minority_upsampled = resample(df_left, 
                                 replace=True,     # sample with replacement
                                 n_samples=2886,    # to match majority class
                                 random_state=27) # reproducible results
 
# Combine majority class with upsampled minority class
df_upsampled = pd.concat([df_stay, df_minority_upsampled])


In [ ]:
#Checking sample
df_upsampled.Churn.value_counts()

In [ ]:
df.dtypes

In [ ]:
df.head(3)

In [ ]:
# Looking at the descriptive statistics of the columns with categorical values
cats = [column for column in df.columns if (df[column].dtype == "O")]
print("Summary table of the Descriptive Statistics of Columns with Numeric Values")
df[cats].describe(include="all")


In [ ]:
# Looking at the descriptive statistics of the columns with numeric values
numerics = [column for column in df.columns if (df[column].dtype != "O")]
print("Summary table of the Descriptive Statistics of Columns with Numeric Values")
df[numerics].describe()


In [ ]:
# Create a boolean mask to identify boolean columns
boolean_mask = df.dtypes == bool

# Select columns with boolean values
boolean_columns = df.columns[boolean_mask]

# Display the selected columns
boolean_columns


#### `Modeling`

In [ ]:
df.dtypes

In [ ]:
X=df.drop(columns=['Churn'],axis=1)
y=df['Churn']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=27)

In [ ]:
for column in X.select_dtypes('object'):
    print(column)

In [ ]:
cats=X.select_dtypes('object').columns
#numerics=X.select_dtypes('number')

##### `Making pipelines`


In [ ]:
scaler = StandardScaler()
encoder = OneHotEncoder(handle_unknown="ignore", sparse_output=False)
# putting numeric columns to scaler and categorical to encoder
num_transformer = Pipeline(steps=[
     ('num_imputer', SimpleImputer(strategy='median')),
    ('num', scaler)
])
cat_transformer = Pipeline(steps=[
   ('cat_imputer', SimpleImputer(strategy='most_frequent')),
    ('cat', encoder)
])

In [ ]:
# getting together our scaler and encoder with preprocessor
preprocessor = ColumnTransformer(
      transformers=[('num', num_transformer , numerics),
                    ('cat', cat_transformer , cats)])

In [ ]:
log_mod= Pipeline(steps=[("preprocessor", preprocessor), 
                          ("model",LogisticRegression(random_state=27 ))])
 

In [ ]:
svc_mod= Pipeline(steps=[("preprocessor", preprocessor), 
                          ("model",SVC() )])
 

In [ ]:
## CatBoost Classifier
CatBoost_mod = Pipeline(steps=[("preprocessor", preprocessor), 
                          ("model", CatBoostClassifier(random_state=27, verbose = False))])


In [ ]:
# Create a dictionary of the model pipelines
all_models_pipeines = {"Logistic_Regressor": log_mod,
              "SVM": svc_mod,
              "CatBoost": CatBoost_mod,
              }
    

In [ ]:
# Create a function to model and return comparative model evaluation scores
# Function to calculate and compare accuracy
def evaluate_models(models=all_models_pipeines, X_test=X_test, y_test=y_test):
    # Key imports
    from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, ConfusionMatrixDisplay

    # Dictionary for trained models
    trained_models = dict()
    
    i = 1
    
    # List to receive scores
    performances = []
    for name, model in models.items():

        # Fit model to training data
        model = model.fit(X_train,  y_train)

        # Predict and calculate performance scores
        y_pred = model.predict(X_test)
        performances.append([name,
                             accuracy_score(y_test, y_pred),  # accuracy
                             precision_score(y_test, y_pred, average="weighted"),  # precisions
                             recall_score(y_test, y_pred,average="weighted"),  # recall
                             f1_score(y_test, y_pred, average="weighted")
                             ])

        # Print classification report
        model_report = classification_report(y_test, y_pred)
        print("This is the classification report of the",name, "model", "\n", model_report, "\n")

        # Defining the Confusion Matrix
        model_conf_mat = confusion_matrix(y_test, y_pred)
        model_conf_mat = pd.DataFrame(model_conf_mat).reset_index(drop=True)
        print(f"Below is the confusion matrix for the {name} model")

        # Visualizing the Confusion Matrix
        f, ax = plt.subplots()
        sns.heatmap(model_conf_mat, annot=True, linewidth=1.0,fmt=".0f", cmap="RdPu", ax=ax)
        plt.xlabel = ("Prediction")
        plt.ylabel = ("Actual")
        plt.show()

        # Store trained model
        trained_model_name = "trained_" + str(name).lower()
        trained_models[trained_model_name] = model
        
        print("\n", "-----   -----"*6, "\n",  "-----   -----"*6)
    
    # Compile accuracy
    df_compare = pd.DataFrame(performances, columns=["model", "accuracy", "precision", "recall", "f1_score"])
    df_compare.set_index("model", inplace=True)
    df_compare.sort_values(by=["f1_score", "accuracy"], ascending=False, inplace=True)
    return df_compare, trained_models

In [ ]:
# Run the function to train models and return performances
all_models_eval, trained_models = evaluate_models()
all_models_eval

In [ ]:
#### HYPERPARAMETER TUNING
#
#
#
X_train

In [124]:
processed = preprocessor.transform(X_train)
#new_data=processed.transform(X_train)
processed

array([[ 1.63768954,  0.9475051 ,  1.        , ...,  1.        ,
         0.        ,  0.        ],
       [ 0.52743657,  1.0139586 ,  1.        , ...,  0.        ,
         0.        ,  0.        ],
       [-0.91178025, -1.06888376,  1.        , ...,  1.        ,
         0.        ,  0.        ],
       ...,
       [-0.45945497, -0.14802812,  0.        , ...,  0.        ,
         1.        ,  0.        ],
       [-1.07626217,  0.25449019,  0.        , ...,  0.        ,
         1.        ,  0.        ],
       [ 0.36295464,  0.06272439,  1.        , ...,  0.        ,
         0.        ,  0.        ]])

In [107]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV

In [130]:
## XGBoost Classifier
xgb_clf = Pipeline(steps=[("preprocessor", preprocessor), 
                          ("model", XGBClassifier(random_state=26))])

# Defining the values for the RandomizedSearchCV
random_grid = {"model__learning_rate": [0.1, 0.3, 0.5, 0.7, 1.0],
               "model__max_depth": [5, 10, 15, 20, 25, 30, 35],
               "model__booster": ["gbtree", "gblinear", "dart"],
               "model__n_estimators": [5, 10, 20, 50, 80, 100]
              }

In [131]:
# Running the RandomizedSearch Cross-Validation with the above set of Parameters
xgb_rs_cv_model = RandomizedSearchCV(estimator=xgb_clf,
                                     param_distributions=random_grid,
                                     n_iter=30,
                                     cv=15,
                                     random_state=26,
                                     n_jobs=-1)

# Fitting the model to the training data
xgb_rs_cv_model.fit(processed,y_train)

ValueError: 
All the 450 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
450 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Python311\Lib\site-packages\sklearn\utils\__init__.py", line 482, in _get_column_indices
    all_columns = X.columns
                  ^^^^^^^^^
AttributeError: 'numpy.ndarray' object has no attribute 'columns'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\Python311\Lib\site-packages\sklearn\model_selection\_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Python311\Lib\site-packages\sklearn\base.py", line 1474, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^

In [128]:
log_mod= Pipeline(steps=[("preprocessor", preprocessor), 
                          ("model",LogisticRegression(verbose=0))])

params= {
    'penalty':['l1', 'l2', 'elasticnet']
}
 

In [129]:
# Running the RandomizedSearch Cross-Validation with the above set of Parameters
log_cv_model = GridSearchCV(param_grid=params,estimator = log_mod) #RandomizedSearchCV(estimator=xgb_clf,
                                    #  param_distributions=random_grid,
                                    #  n_iter=30,
                                    #  cv=15,
                                    #  random_state=26,
                                    #  n_jobs=-1)
log_cv_model.fit(processed, y_train)
# Fitting the model to the training da

ValueError: Invalid parameter 'penalty' for estimator Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('num_imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('num',
                                                                   StandardScaler())]),
                                                  ['tenure', 'MonthlyCharges']),
                                                 ('cat',
                                                  Pipeline(steps=[('cat_imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('cat',
                                                                   OneHotEncoder(handle_unknown='ignore',
 

In [ ]:
log_mod.fit(X_train,  y_train)


In [ ]:

y_pred = log_mod.predict(X_test)
model_conf_mat = confusion_matrix(y_test, y_pred)

In [ ]:
model_conf_mat

In [ ]:
X_train.shape

In [ ]:

from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

# define model and parameters
model = SVC()
kernel = ['poly', 'rbf', 'sigmoid']
C = [50, 10, 1.0, 0.1, 0.01]
gamma = ['scale']
# define grid search
grid = dict(kernel=kernel,C=C,gamma=gamma)
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
grid_search = GridSearchCV(estimator=model, param_grid=grid, n_jobs=-1, cv=cv, scoring='accuracy',error_score=0)
grid_result = grid_search.fit(X_train, y_train)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))